# Looking for food in the blind

Day after day, Dr. Vasilescu dedicated herself to constructing a sustainable habitat out of the remnants of the damaged Artemis spacecraft. She also tried to set up her robotic companions so that they work tirelessly alongside her, so that their mechanical precision compensates for the absence of human interaction. The first system that she managed to bring back online is a camera placed on the initial landing site, which is away from her current habitat and may give her indications on where to find useful resources for her survival.

<img src="../artwork/stranded/mars_landing_site_with_base.png" width=60%/>

She could get readings from the camera about the position of a very important package with food supplies, however she needed to understand where exactly is the package relative to her base. She had to understand the basics of coordinate frames and transformations.

## Coordinate frames
Kinematics is the study of motions of objects in space. This is very relevant when working with robots, since we are usually interested in interacting with objects in space, or with positioning sensors in specific poses (position and orientation) so that they can gather useful information (e.g., getting images). How do we describe kinematics?

To describe pose (position and orientation) of objects, we use the concept of _coordinate frames_ (or as you might have learned it in the past, _coordinate systems_). When dealing with moving objects for which we need to describe their _kinematics_, we always need a reference on which distances should be reported on. This is essential for any type of robot, whether it is a robot arm, mobile robot, drone, or underwater. If, besides position, we also care about orientation, then we need to use two coordinate frames: a _reference_ frame, and a _moving_ or _target_ frame.

A coordinate frame consists of axes, on top of which we are measuring distances. The number of axes depends on the dimensions of the motion we are studying. For instance, if we are studying the motion of a body moving in one dimension, then one single axis would be enough. If we study motions of objects on flat surfaces (e.g., mobile robots), we use coordinate frames with two axes. If we study motions of objects in three-dimensional (3D) space (e.g., robot arms, drones, underwater robots), then we use coordinate frames with three axes. A necessary condition for defining an orthogonal coordinate frame, is that its axes must be all perpendicular to each other. When the units of distance on each of the axes are equal as well, then we say that this coordinate frame is orthonormal.

When defining coordinate frames in three (or more dimensions), the position of the axes is important. For three dimensions, coordinate frames must respect the rule of the right hand, which is demonstrated in the figure below. To make things more visual and simple, we are also usually coloring each axis. The usual convention in robotics is that $X$ is red, $Y$ is green, and $Z$ is blue. We will use this convention for all coordinate frames in this course.

<img src="../artwork/transformations/R3-right_hand_rule.png" width=30%/>

A coordinate frame is essentially a collection of three vectors. The way we represent vectors in mathematical terms is using the vector notation of a point in space. The vector has three elements, each representing the coordinates of the end-point of the vector in a reference coordinate frame:

$$
V_1  = \begin{bmatrix}
   P_x \\
   P_y \\
   P_z \\
\end{bmatrix}
$$

To represent a coordinate frame, we group three vectors together in order to form a matrix:

$$
V  = \begin{bmatrix}
   P_x^X & P_y^Y & P_x^Z \\
   P_y^X & P_y^Y & P_y^Z \\
   P_z^X & P_z^Y & P_z^Z \\
\end{bmatrix}
$$

or simpler:

$$
V  = \begin{bmatrix}
   X_x & Y_y & Z_x \\
   X_y & Y_y & Z_y \\
   X_z & Y_z & Z_z \\
\end{bmatrix}
$$

where $P_x^Y$ is the $x$ coordinate of the $Y$ axis. This is called the __transformation matrix__ and it tells us how is a _target_ coordinate frame __oriented__ relative to a _reference_ frame. In order to calculate it, we assume that the origin of the vectors are at the origin of the reference frame, and that they all have length 1. Since we always need to express a _target_ coordinate frame w.r.t (with respect to) a _reference_ frame, we need to use appropriate notation. We denote $T_r^t$ the transformation matrix of frame $t$ w.r.t frame $r$. This is called a [_3D rotation group_ and is often denoted as __SO(3)__](https://en.wikipedia.org/wiki/3D_rotation_group).

You can visualize how the numbers in this matrix are changing when changing the orientation of a target frame $t$ w.r.t. a reference frame $r$, in the interactive widget below.

In [ ]:
from ipywidgets import interact
from spatialmath.base import *

def showAxes(x=0, y=0, z=0):
    t = rotx(x*3.14/180)@roty(y*3.14/180)@rotz(z*3.14/180)
    trplot(transl(0,0,0), color='gray', width=1, frame='r', length=1.3)
    trplot(t, frame='t', style='rviz', width=2, dims=[-1, 1, -1, 1, -1, 1])
    print(t)

interact(showAxes, x=(-90,90), y=(-90,90), z=(-90,90))

## Transformations

As demonstrated by the previous widget, a _transformation matrix_ represents a rotated coordinate frame with respect to a reference frame. The three columns of the matrix represent the coordinates of the axes of the _transformed_ frame w.r.t. the reference frame. Each of the three axes has three coordinates, and that is how we end up with a $3 \times 3$ matrix. At the same time, _transformation matrices_ can be thought of as __actions__ for transforming an existing frame. You can imagine that you have a _target frame_ that is perfectly aligned with the reference frame. In that case, the _transformation matrix_ that represents the _target frame_ is:

$$
V  = \begin{bmatrix}
   X_x & Y_y & Z_x \\
   X_y & Y_y & Z_y \\
   X_z & Y_z & Z_z \\
\end{bmatrix} = \begin{bmatrix}
   1 & 0 & 0 \\
   0 & 1 & 0 \\
   0 & 0 & 1 \\
\end{bmatrix} 
$$

which is the $3 \times 3$ identity matrix. If we want to _transform_ this frame, we can multiply with another _transformation matrix_. Since the initial frame is the identity matrix, the result of this multiplication will be equal to the _transformation matrix_. Therefore, a transformation matrix represents __both a transformed frame__ w.r.t. to a reference frame, but also __the action of transforming__ a frame w.r.t. a reference frame.

### Rotation

When dealing with coordinate frames represented by $3 \times 3$ matrices, we have only information about the orientation of the frames. Therefore, the only kind of transformations that we can perform (preserving orthogonality) are rotations. Since we have 3 axes, we can perform 3 types of rotations, each around one of the axes. You can visualize these three rotations using the widget below

In [ ]:
from ipywidgets import interact, widgets
from spatialmath.base import *
import numpy as np

def RotateOnX(r):
    t = rotx(r)
    trplot(transl(0,0,0), color='gray', width=1, frame='r', length=1.3)
    trplot(t, frame='t', style='rviz', width=2, dims=[-1, 1, -1, 1, -1, 1])
    print(t)

def RotateOnY(p):
    t = roty(p)
    trplot(transl(0,0,0), color='gray', width=1, frame='r', length=1.3)
    trplot(t, frame='t', style='rviz', width=2, dims=[-1, 1, -1, 1, -1, 1])
    print(t)

def RotateOnZ(y):
    t = rotz(y)
    trplot(transl(0,0,0), color='gray', width=1, frame='r', length=1.3)
    trplot(t, frame='t', style='rviz', width=2, dims=[-1, 1, -1, 1, -1, 1])
    print(t)

outx = widgets.interactive(RotateOnX, r=(-3.14,3.14))
outy = widgets.interactive(RotateOnY, p=(-3.14,3.14))
outz = widgets.interactive(RotateOnZ, y=(-3.14,3.14))

tab  = widgets.Tab(children = [outx, outy, outz])
tab.set_title(0, 'Rot X')
tab.set_title(1, 'Rot Y')
tab.set_title(2, 'Rot Z')

display(tab)

What is the mathematical expression of these transformations? If you observe closely the numbers in the matrices, you will notice a certain pattern: some numbers don't change, and it seems to be those relating to the axis that the rotation is happening about. This makes sense, since e.g. when we rotate around the $X$ axis, the coordinates of that axis are not changing w.r.t. the reference frame.

The general form of the transformation matrices relating to rotations around each of the three axes is the following:

Rotation around $x$ axis:
$Rotx(\varphi) = \begin{bmatrix}
   1 & 0 & 0  \\
   0 & {\cos \varphi} & { - \sin \varphi }  \\
   0 & {\sin \varphi } & {\cos \varphi}  \\
\end{bmatrix}$

Rotation around $y$ axis:
$Roty(\vartheta) = \begin{bmatrix}
   {\cos \vartheta } & 0 & {\sin \vartheta }  \\
   0 & 1 & 0  \\
   { - \sin \vartheta } & 0 & {\cos \vartheta }  \\
\end{bmatrix}$

Rotation around $z$ axis:
$Rotz(\psi) = \begin{bmatrix}
   {\cos \psi } & { - \sin \psi } & 0  \\
   {\sin \psi } & {\cos \psi } & 0  \\
   0 & 0 & 1  \\
\end{bmatrix}$

### Translation

As you observed, the above transformation matrices affect only the orientation of the target coordinate frame. However, coordinate frames can also be displaced/translated w.r.t. a reference frame. To take translations into consideration, we need to use [__Homogeneous coordinates__](https://en.wikipedia.org/wiki/Homogeneous_coordinates). Using homogeneous coordinates, we can represent any vector in 3D space using 3 coordinates and a scaling factor. When we need to keep the same scale when applying a transformation, the scaling factor is one. Therefore, a point in space is represented as following:

$$ P = \begin{bmatrix} P_x \\ P_y \\ P_z \\ 1 \end{bmatrix} $$

In this case, transformations are defined as $4 \times 4$ matrices of the following structure:

$$
T = \begin{bmatrix}
   {} & {} & {} & {}  \\
   3 &  \times  & 3 & {3 \times 1}  \\
   {} & {} & {} & {}  \\ \hline
   1 &  \times  & 3 & {1 \times 1}  \\
\end{bmatrix} =
\begin{bmatrix}
   {} & {} & {} & {trans - }  \\
   {} & {rotation} & {} & {la - }  \\
   {} & {} & {} & {tion}  \\ \hline
   0 & 0 & 0 & 1  \\
\end{bmatrix} =
\begin{bmatrix}
   X_x & Y_x & Z_x & P_x \\
   X_y & Y_y & Z_y & P_y \\
   X_z & Y_z & Z_z & P_z \\
   0   & 0   & 0   & 1
\end{bmatrix}
$$

The rotation $ 3 \times 3 $ part of the matrix, represents as before the orientation of the axes of the _target_ frame w.r.t. the reference frame. The translation $ 3 \times 1 $ vector, represents the position of the origin of the _target_ frame w.r.t. the reference frame. The last row has always fixed values of 0s and 1. Using homogeneous transformation matrices, we can define the following elementary transformations:

Translation on $X$ axis: $TransX(a) = \begin{bmatrix}
   1 & 0 & 0 & a  \\
   0 & 1 & 0 & 0  \\
   0 & 0 & 1 & 0  \\
   0 & 0 & 0 & 1  \\
\end{bmatrix} $

Translation on $Y$ axis: $TransY(b) = \begin{bmatrix}
   1 & 0 & 0 & 0  \\
   0 & 1 & 0 & b  \\
   0 & 0 & 1 & 0  \\
   0 & 0 & 0 & 1  \\
\end{bmatrix} $

Translation on $Z$ axis: $TransZ(c) = \begin{bmatrix}
   1 & 0 & 0 & 0  \\
   0 & 1 & 0 & 0  \\
   0 & 0 & 1 & c  \\
   0 & 0 & 0 & 1  \\
\end{bmatrix} $

Rotation around $X$ axis: $RotX(\vartheta) = \begin{bmatrix}
   1 & 0 & 0 & 0  \\
   0 & {\cos \vartheta } & { - \sin \vartheta } & 0  \\
   0 & {\sin \vartheta } & {\cos \vartheta } & 0  \\
   0 & 0 & 0 & 1  \\
\end{bmatrix}$

Rotation around $Y$ axis: $RotY(\varphi) = \begin{bmatrix}
   {\cos \varphi } & 0 & {\sin \varphi } & 0  \\
   0 & 1 & 0 & 0  \\
   { - \sin \varphi } & 0 & {\cos \varphi } & 0  \\
   0 & 0 & 0 & 1  \\
\end{bmatrix} $

Rotation around $Z$ axis: $ RotZ(\psi ) = \begin{bmatrix}
   {\cos \psi } & { - \sin \psi } & 0 & 0  \\
   {\sin \psi } & {\cos \psi } & 0 & 0  \\
   0 & 0 & 1 & 0  \\
   0 & 0 & 0 & 1  \\
\end{bmatrix} $

As before, a transformation matrix can be understood in different ways:

- The action of transforming a coordinate frame into a new pose (position and orientation)
- A description of the pose (position and orientation) of a _target_ coordinate system w.r.t a _reference_ coordinate system

## Combination of transformations

The transformations we saw above are all elementary, in the sense that they describe either a translation or a rotation on a single axis. However, frames can be arbitrarily oriented relative to each other, resulting in more complex transformations? To describe such complex transformations, we need to combine elementary transformations through multiplication.

> Any arbitrary transformation between two frames can be described with a finite sequence of elementary transformations

Since transformation matrices are $4 \times 4$, a multiplication between two such matrices is possible and will result into another $4 \times 4$ matrix. This new matrix will also fulfil the conditions for being a transformation matrix. You can verify that through matrix multiplication, the last row of the resulting matrix will still have the same form of a homogeneous transformation matrix.

$$
AB = 
\begin{bmatrix}
   AX_x & AY_x & AZ_x & AP_x \\
   AX_y & AY_y & AZ_y & AP_y \\
   AX_z & AY_z & AZ_z & AP_z \\
   0   & 0   & 0   & 1
\end{bmatrix} \begin{bmatrix}
   BX_x & BY_x & BZ_x & BP_x \\
   BX_y & BY_y & BZ_y & BP_y \\
   BX_z & BY_z & BZ_z & BP_z \\
   0   & 0   & 0   & 1
\end{bmatrix} = 
\begin{bmatrix}
   CX_x & CY_x & CZ_x & CP_x \\
   CX_y & CY_y & CZ_y & CP_y \\
   CX_z & CY_z & CZ_z & CP_z \\
   0   & 0   & 0   & 1
\end{bmatrix}
= C
$$

We can therefore combine several elementary transformations through multiplication to generate more complex transformations. A subtle detail comes in the order of multiplication, since as we know from linear algebra:

$$ AB \neq BA $$

Which way do we need to multiply the transformations then when we are combining them?

### Absolute transformation (Left multiplication)

Considering an initial transformation matrix $T_B^A$ describing the pose of frame A w.r.t. frame B, we can further __transform frame A about the axes of frame B__, by multiplying from the __left__ with a transformation matrix $C$. You can see a visualization of an _absolute_ transformation in the following widget. Notice how frame A, which is already transformed w.r.t. frame B, is further transformed about the $X$ axis coordinate frame B, both in the translation and rotation tab.

In [ ]:
from ipywidgets import interact, widgets
from spatialmath.base import *
import numpy as np

TbA = transl(0.5,0.5,1)@trotz(0.807)

def TranslateOnXLeft(a):
    t = transl(a,0,0)@TbA
    trplot(trotx(0), color='gray', width=1, frame='B', length=1.3)
    trplot(t, frame='A', style='rviz', width=2, dims=[-2, 2, -2, 2, -2, 2])
    print('C*T = \n',t)
    
def RotateOnXLeft(a):
    t = trotx(a)@TbA
    trplot(trotx(0), color='gray', width=1, frame='B', length=1.3)
    trplot(t, frame='A', style='rviz', width=2, dims=[-2, 2, -2, 2, -2, 2])
    print('C*T = \n',t)

rotx = widgets.interactive(RotateOnXLeft, a=(-3.14,3.14))
transx = widgets.interactive(TranslateOnXLeft, a=(-2.0,2.0))

tab  = widgets.Tab(children = [rotx, transx])
tab.set_title(0, 'rotation')
tab.set_title(1, 'translation')

display(tab)

### Relative transformation (Right multiplication)

Considering an initial transformation matrix $T_B^A$ describing the pose of frame A w.r.t. frame B, we can further __transform frame A about the axes of frame A__, by multiplying from the __right__ with a transformation matrix $C$. You can see a visualization of a _relative_ transformation in the following widget. Notice how frame A, which is already transformed w.r.t. frame B, is further transformed about its own $X$ axis, both in the translation and rotation tab.

In [ ]:
from ipywidgets import interact, widgets
from spatialmath.base import *
import numpy as np

TbA = transl(0.5,0.5,1)@trotz(0.807)

def TranslateOnXRight(a):
    t = TbA@transl(a,0,0)
    trplot(trotx(0), color='gray', width=1, frame='B', length=1.3)
    trplot(t, frame='A', style='rviz', width=2, dims=[-2, 2, -2, 2, -2, 2])
    print('T*C = \n',t)
    
def RotateOnXRight(a):
    t = TbA@trotx(a)
    trplot(trotx(0), color='gray', width=1, frame='B', length=1.3)
    trplot(t, frame='A', style='rviz', width=2, dims=[-2, 2, -2, 2, -2, 2])
    print('C*T = \n',t)

rotx = widgets.interactive(RotateOnXRight, a=(-3.14,3.14))
transx = widgets.interactive(TranslateOnXRight, a=(-2.0,2.0))

tab  = widgets.Tab(children = [rotx, transx])
tab.set_title(0, 'rotation')
tab.set_title(1, 'translation')

display(tab)

## Switching frames

Quite often in robotic applications, we need to transform a known pose from one frame into another. Let's say we have pose data for an object from a camera, which is reporting poses in its own intrinsic coordinate frame. However, we want to interact with this object using a tool placed at the tip of a robot, which is calculated in the coordinate frame of the robot. Using transformation matrices, we can transform a known pose $T_r^A$ w.r.t. to _reference_ frame $r$, into a pose $T_t^A$ w.r.t. a _target_ frame, as long as we know the transformation $T_t^r$ between the _target_ and _reference_ frame. This is possible through the following operation:

$$ T_t^A = T_t^r  T_r^A $$

If we don't know transformation $T_t^r$, and instead we know the transformation $T_r^t$ from _reference_ to _target_, we can __invert__ the transformation by inverting the matrix:

$$ T_r^t = (T_t^r)^{-1} $$

it can be shown that for homogeneous transformation matrices, the inverse of matrix T has the following form:

$$
T^{ - 1} = \begin{bmatrix}
 {X_X } & {X_Y } & {X_Z } & { - P_X\cdot X_X - P_Y \cdot X_Y - P_Z\cdot X_Z }  \\
 {Y_X } & {Y_Y } & {Y_Z } & { - P_X \cdot Y_X - P_Y \cdot Y_Y - P_Z\cdot Y_Z }  \\
 {Z_X } & {Z_Y } & {Z_Z } & { - P_X\cdot Z_X - P_Y \cdot Z_Y - P_Z\cdot Z_Z }  \\
 0  & 0 & 0  & 1
\end{bmatrix}
$$

## Spatial math commands

There are several toolboxes available online for helping us with various robotics operations. The one we will use extensively during the laboratories is [__spatialmath__](https://bdaiinstitute.github.io/spatialmath-python/) together with its __spatialmath.base__. This toolbox offers a very easy way to calculate various transformation matrices, using the __trotx, troty, trotz, and transl__ commands. You can see an example of how are these used below:

In [ ]:
from spatialmath.base import *
from spatialmath import *

t1 = trotx(45, 'deg')
t2 = trotz(45, 'deg')
t3 = transl(2, 0 ,-1)

# print is a function in python 3, it needs brackets
print(t2)

# The matrices obtained using this method can be combined to calculate more complex transformations by multiplication:

# Calculate coordinate '3' frame after three consecutive transformations
t13 = t1@t2@t3
print(t13)

In [ ]:
from spatialmath.base import *
from spatialmath import *

# You can also visualise the coordinate frame if you use the 'SE3' class
p1 = SE3.Rx(theta=45, unit='deg')
p3 = SE3.Ry(theta=45, unit='deg')
p2 = SE3(x=2, y=0, z=-1)

(p1@p2@p3).plot()

We can also define a point w.r.t. a coordinate frame, and find out what are its coordinates on another frame if we know the transformation between the two frames. For defining the point, we use the [__numpy__](https://numpy.org/) library. By default, numpy creates row vectors, that's why we need to transpose the vector into a column before we do the multiplication. We do that using the __T__ method:

In [ ]:
import numpy as np

# Define point P with coordinates 1,2,3
P3 = np.array([[1,2,3,1]]).T

# Calculating coordinates of point P w.r.t. frame '1', if we know its coordinates w.r.t. frame 3
t13@P3

## Helping dr. Vasilescu

Now that you know enough about coordinate frames and transformations, go ahead and help dr. Vasilescu [find necessary resources for survival](../assignments/Assignment01_Transformations.ipynb)